# Pipelining with Titanic Data

### Data loading ...

In [56]:
from functools import lru_cache
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import fcluster, single, complete
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from scipy.stats import randint, uniform 
from collections.abc import Iterable
from mlpipes.pfunc import get_ohe
import pandas as pd
import re
import warnings
import numpy as np
from mlpipes.pfunc import *
from mlpipes.utils import iterate_over_pars
import cachepy

train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
combined = pd.concat([train, test]).reset_index(drop=True)

/home/dmitry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [57]:
cache = cachepy.Cache()

#### Some preprocessing of the data

## Utility functions

In [58]:
def parse_name(s): 
    a, b = s.split(',')
    family_name = a.strip()
    title = b.split('.')[0].strip()
    first_name = b.split('.')[1].split()[0].strip()
    return (first_name.replace('(', '').replace(')', ''), title, family_name)

def parse_cabin_letter(column):
    letter_pat = re.compile('([A-Za-z])\d+')
    return list(map(lambda x: letter_pat.findall(str(x))[0] if letter_pat.findall(str(x)) else pd.np.nan, column.values.tolist()))

def parse_ticket_number(column):
    number_pat = re.compile('\d{3,}')
    numbers = map(lambda x: number_pat.findall(x)[0] if number_pat.findall(x) else pd.np.nan, column)
    return pd.Series(numbers)

def get_friendship_group(df):
    friendship_group_counter = 0
    if 'family_name' not in df.columns:
        family_names = pd.Series(map(lambda x: parse_name(x)[-1], df.Name))
    else:
        family_names = df.family_name
    cabins = pd.Series(map(parse_cabin_letter, df.Cabin))
    ticket_grouping = []
    for family, count in family_names.value_counts().items():
        family_mask = family_names == family
        
        if count == 1:
            ticket_grouping.append(friendship_group_counter)
            friendship_group_counter += 1
            continue

In [59]:
@cache
def get_family_name(df):
    family_names = pd.Series(map(lambda x: parse_name(x)[-1], df.Name))
    df_ = df.copy()
    df_.loc[:,'family_name'] = family_names
    return df_

@cache
def get_ticket_group(df):
    df_ = df.copy()
    grouped = df.Ticket.groupby(parse_ticket_number(df.Ticket))
    for ind, key in enumerate(grouped.indices):
        df_.loc[grouped.indices[key], 'ticket_group'] = ind if len(grouped.indices[key])>1 else -1
    return df_

@cache
def get_cabin_letter(df):
    df_ = df.copy()
    cabins = parse_cabin_letter(df.Cabin)
    df_.loc[:, 'cabin_na'] = pd.isnull(df.Cabin)
    df_.loc[:, 'cabin'] = cabins
    return df_

@cache
def get_is_alone(df):
    df_ = df.copy()
    df_.loc[:, 'is_alone'] = (df.loc[:, 'Parch'] + df.loc[:, 'SibSp'] + 1 == 1).astype(int)
    return df_

@cache
def get_family_size(df):
    df_ = df.copy()
    df_.loc[:, 'family_size'] = df_.loc[:, 'Parch'] + df_.loc[:, 'SibSp'] + 1
    return df_

@cache
def get_titles(df):
    df_ = df.copy()
    titles = pd.Series(map(lambda x: parse_name(x)[1], df.Name))
    df_.loc[:, 'title'] = titles
    return df_

@cache
def discretize_faries(df, ngroups=3):
    df_ = df.copy()
    df_.loc[:, 'fares'] = pd.cut(df_.loc[:,'Fare'], ngroups, labels=False)
    return df_

@cache
def discretize_ages(df, ngroups=3):
    df_ = df.copy()
    df_.loc[:, 'Age'] = pd.cut(df_.loc[:,'Age'], ngroups, labels=False)
    return df_

@cache
def estimate_age(df):
    estimates = []
    for ind, row in df.loc[df.Age.isnull(), :].iterrows():
        # NOTE: Could be rewritten using vectorized notation
        if row.title in ['Master', 'Mr', 'Miss', 'Rev', 'Dr']:
            estimates.append(df.groupby(['title', 'Sex']).median().loc[[row.title, row.Sex],'Age'].values[0])
        else:
            estimates.append(df.groupby(['Sex', 'Pclass']).median().loc[[row.Sex], 'Age'].values[0])
    df_ = df.copy()
    df_.loc[df.Age.isnull(), 'Age'] = estimates
    return df_

@cache
def get_cabin_groups(df):
    num_pat = re.compile('\d+')
    let_pat = re.compile('[a-zA-Z]')
    LONG_DISTANCE = 5
    MEDIUM_DISTANCE = 4
    NORMAL_DISTANCE = 3
    SMALL_DISTANCE = 2
    LOW_DISTANCE = 1
    EQUAL = 0
    def cabin_distance(u, v):
        _u, _v = u[0], v[0]
        if not isinstance(_u, Iterable) or not isinstance(_v, Iterable):
            return LONG_DISTANCE
        unums = list(map(int, sum(map(num_pat.findall, _u), [])))
        vnums = list(map(int, sum(map(num_pat.findall, _v), [])))
        ulets = list(sum(map(let_pat.findall, _u), []))
        vlets = list(sum(map(let_pat.findall, _v), []))
        if not(unums and vnums):
            if set(ulets).intersection(vlets):
                return EQUAL
            else:
                return MEDIUM_DISTANCE
        if u == v:
            return EQUAL
        if set(_u).intersection(set(_v)):
            return LOW_DISTANCE
        if not set(ulets).intersection(set(vlets)):
            return MEDIUM_DISTANCE
        else:
            for p in _u:
                for q in _v:
                    try:
                        pval = list(map(int, num_pat.findall(p)))[0]
                        qval = list(map(int, num_pat.findall(q)))[0]
                        if p[0] == q[0] and (abs(pval - qval) <= 2):
                            return SMALL_DISTANCE
                    except IndexError:
                        pass
            return NORMAL_DISTANCE
        return MEDIUM_DISTANCE
    distances = pdist(df.Cabin.apply(lambda x: x.split() if not isinstance(x, float) else x).values[:, np.newaxis], cabin_distance)
    df_ = df.copy()
    df_.loc[:, 'cabin_group'] = fcluster(complete(distances), SMALL_DISTANCE, criterion='distance')
    df_.cabin_group = df_.groupby('cabin_group')['cabin_group'].transform(lambda x: x if len(x)>1 else pd.Series([-1]*len(x)))
    return df_

@cache
def combine_titles(df):
    df_ = df.copy()
    df_['title'] = df_['title'].replace(['Mlle'], 'Miss')
    df_['title'] = df_['title'].replace(['Ms'], 'Miss')
    df_['title'] = df_['title'].replace(['Mme'], 'Mrs')
    df_['title'] = df_['title'].replace(['Lady', 'the Countess', 'Capt', 'Col',
                                         'Don', 'Dr', 'Major', 'Rev', 'Sir',
                                         'Jonkheer', 'Dona'], 'rare')
    return df_

@cache
def simple_encoder(df):
    df_ = df.copy()
    sex_mapping = {'male': 0, 'female':1}
    embarked_mapping = {'S':0, 'Q':1, 'S':2}
    df_.Embarked = df_.Embarked.map(embarked_mapping)
    df_.Sex = df_.Sex.map(sex_mapping)
    return df_

@cache
def label_encode(df, **kwargs):
    return get_le(df, **kwargs)[0]

## Building pipelines

In [66]:
preprocessing_pipeline = (('add_ticket_group', get_ticket_group, {}),
                          ('add_family_size', get_family_size, {}),
                          ('add_titles', get_titles, {}),
                          ('convert_fares', discretize_faries, {'ngroups': 3}),
                          ('fill_embarked', fill_na_simple, {'colnames': ('Embarked',),
                                                             'methods': (lambda x: pd.Series(x).mode()[0],)}),
                          ('add_family_name', get_family_name, {}),
                          ('add_ages', estimate_age, {}),
                          ('convert_ages', discretize_ages, {'ngroups': 5}),
                          ('add_cabin_groups', get_cabin_groups, {}),
                          ('combine_titles', combine_titles, {}),
                          ('sex_encoder', simple_encoder, {}),
                          ('drop_columns', drop_columns, {'colnames': ('Survived',
                                                                       'PassengerId',
                                                                       'SibSp',
                                                                       'Parch',
                                                                       'Ticket',
                                                                       'Fare',
                                                                       'Name',
                                                                       'Cabin',
                                                                       )}),
                          ('get_le', label_encode, {'colnames': ('ticket_group', 'cabin_group',
                                                                 'title', 'Embarked', 'family_name')})
                         )

def process(pipeline, data, parameters=dict()):
    data_ = data.copy()
    for name, func, kwargs in pipeline:
        if name in parameters:
            kwargs = parameters[name]
        data_ = func(data_, **kwargs)
    return data_


# Preprocessing steps (feature engeneering)

In [67]:
processed  = process(preprocessing_pipeline, combined)

In [68]:
processed.head()

,Age,Pclass,Sex,family_size,fares,LE_ticket_group,LE_cabin_group,LE_title,LE_Embarked,LE_family_name
0,1,3,0,2,0.0,0,0,2,1,100
1,2,1,1,2,0.0,63,9,3,6,182
2,1,3,1,1,0.0,0,0,1,1,329
3,2,1,1,2,0.0,20,21,3,1,267
4,2,3,0,1,0.0,0,0,2,1,15


In [63]:
parameters_GB = {'n_estimators': randint(20, 300),
                 'max_depth': randint(3, 20),
                 'subsample': uniform(0.7, 0.3),
                 'learning_rate': uniform(0.0001, 0.3),
                 'max_features': ('auto', 'log2', None),
                 'min_samples_leaf': randint(3, 10),
                 'min_samples_split' : randint(2, 10)
                }
preprocessing_parameters = {'convert_ages': {'ngroups': [2,3,4,5]},
                            'convert_fares': {'ngroups': [2,3,4,5]}
                           }
clf = GradientBoostingClassifier(random_state=42)
y = train.Survived.values

In [ ]:
for par in iterate_over_pars(preprocessing_parameters):
    print("=" * 40)
    print("Processing par: ", par)
    processed  = process(preprocessing_pipeline, combined, parameters=par)
    X = processed.iloc[:train.shape[0]].values
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        clfcv = RandomizedSearchCV(clf, param_distributions=parameters_GB, n_iter=1000, scoring='f1',
                               cv=3, verbose=1, n_jobs=-1)
        clfcv.fit(X, y)
    
    cross_val_score(clfcv.best_estimator_, X, y, cv=5, scoring='f1')
    clfcv.best_score_
    print("=" * 40)






Processing par:  {'convert_ages': {'ngroups': 2}, 'convert_fares': {'ngroups': 2}}
Fitting 3 folds for each of 1000 candidates, totalling 3000 fits


[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 358 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done 608 tasks      | elapsed:   20.1s
[Parallel(n_jobs=-1)]: Done 958 tasks      | elapsed:   32.6s
[Parallel(n_jobs=-1)]: Done 1408 tasks      | elapsed:   48.0s
[Parallel(n_jobs=-1)]: Done 1958 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 2608 tasks      | elapsed:  1.5min


In [55]:
raise StopIteration

StopIteration: 

# Simple Neural Network 

In [21]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import backend as K
from keras.utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn import datasets
iris = datasets.load_iris()


BATCH_SIZE = 15
NUM_CLASSES = 2
INPUT_SHAPE = processed.shape[1]
EPOCHS = 2000

# ------------- Building the model ----------------
model = Sequential()
model.add(Dense(20, input_shape=(INPUT_SHAPE,), activation='relu'))
model.add(Dense(100, activation='elu'))
model.add(Dropout(0.1))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(30, activation='elu'))
model.add(Dropout(0.1))
model.add(Dense(20, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10, activation='elu'))
model.add(Dense(NUM_CLASSES-1, activation='sigmoid'))
# -------------------------------------------------


# ---- Building the training and test data --------
X = processed.iloc[:train.shape[0]].values
Y = train.Survived.values
#x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42, stratify=Y)

#y_train = keras.utils.to_categorical(y_train, 3)
#y_test = keras.utils.to_categorical(y_test, 3)
# -------------------------------------------------


In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 20)                200       
_________________________________________________________________
dense_37 (Dense)             (None, 100)               2100      
_________________________________________________________________
dropout_18 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_19 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_39 (Dense)             (None, 30)                1530      
_________________________________________________________________
dropout_20 (Dropout)         (None, 30)                0         
__________

In [23]:
# ----------- Fitting the model -------------------
model.compile(keras.optimizers.Adagrad(), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X, Y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          validation_split=0.3)
# -------------------------------------------------

Train on 623 samples, validate on 268 samples
Epoch 1/2000
623/623 [==============================] - 1s 1ms/step - loss: 0.8193 - acc: 0.6132 - val_loss: 0.6260 - val_acc: 0.6679
Epoch 2/2000
623/623 [==============================] - 0s 245us/step - loss: 0.6879 - acc: 0.5987 - val_loss: 0.5864 - val_acc: 0.6716
Epoch 3/2000
623/623 [==============================] - 0s 293us/step - loss: 0.6329 - acc: 0.6453 - val_loss: 0.5571 - val_acc: 0.7351
Epoch 4/2000
623/623 [==============================] - 0s 204us/step - loss: 0.6085 - acc: 0.6404 - val_loss: 0.5699 - val_acc: 0.6828
Epoch 5/2000
623/623 [==============================] - 0s 241us/step - loss: 0.6225 - acc: 0.6180 - val_loss: 0.5484 - val_acc: 0.6978
Epoch 6/2000
623/623 [==============================] - 0s 345us/step - loss: 0.5864 - acc: 0.6742 - val_loss: 0.5451 - val_acc: 0.7164
Epoch 7/2000
623/623 [==============================] - 0s 260us/step - loss: 0.5987 - acc: 0.6629 - val_loss: 0.5452 - val_acc: 0.6791
Epoc

Epoch 120/2000
623/623 [==============================] - 0s 229us/step - loss: 0.5399 - acc: 0.7207 - val_loss: 0.5418 - val_acc: 0.7388
Epoch 121/2000
623/623 [==============================] - 0s 209us/step - loss: 0.5228 - acc: 0.7255 - val_loss: 0.5443 - val_acc: 0.7388
Epoch 122/2000
623/623 [==============================] - 0s 260us/step - loss: 0.5209 - acc: 0.7480 - val_loss: 0.5414 - val_acc: 0.7425
Epoch 123/2000
623/623 [==============================] - 0s 246us/step - loss: 0.5201 - acc: 0.7287 - val_loss: 0.5392 - val_acc: 0.7388
Epoch 124/2000
623/623 [==============================] - 0s 241us/step - loss: 0.5198 - acc: 0.7287 - val_loss: 0.5421 - val_acc: 0.7388
Epoch 125/2000
623/623 [==============================] - 0s 243us/step - loss: 0.5301 - acc: 0.7319 - val_loss: 0.5402 - val_acc: 0.7500
Epoch 126/2000
623/623 [==============================] - 0s 257us/step - loss: 0.5247 - acc: 0.7400 - val_loss: 0.5392 - val_acc: 0.7500
Epoch 127/2000
623/623 [==========

623/623 [==============================] - 0s 273us/step - loss: 0.5041 - acc: 0.7544 - val_loss: 0.5277 - val_acc: 0.7575
Epoch 239/2000
623/623 [==============================] - 0s 262us/step - loss: 0.5000 - acc: 0.7448 - val_loss: 0.5239 - val_acc: 0.7537
Epoch 240/2000
623/623 [==============================] - 0s 266us/step - loss: 0.4898 - acc: 0.7608 - val_loss: 0.5254 - val_acc: 0.7537
Epoch 241/2000
623/623 [==============================] - 0s 266us/step - loss: 0.4892 - acc: 0.7608 - val_loss: 0.5232 - val_acc: 0.7537
Epoch 242/2000
623/623 [==============================] - 0s 336us/step - loss: 0.4968 - acc: 0.7624 - val_loss: 0.5226 - val_acc: 0.7537
Epoch 243/2000
623/623 [==============================] - 0s 249us/step - loss: 0.4930 - acc: 0.7528 - val_loss: 0.5188 - val_acc: 0.7575
Epoch 244/2000
623/623 [==============================] - 0s 269us/step - loss: 0.4973 - acc: 0.7448 - val_loss: 0.5254 - val_acc: 0.7537
Epoch 245/2000
623/623 [=========================

623/623 [==============================] - 0s 256us/step - loss: 0.4744 - acc: 0.7865 - val_loss: 0.5023 - val_acc: 0.7799
Epoch 357/2000
623/623 [==============================] - 0s 254us/step - loss: 0.4725 - acc: 0.7721 - val_loss: 0.5040 - val_acc: 0.7724
Epoch 358/2000
623/623 [==============================] - 0s 243us/step - loss: 0.4661 - acc: 0.7769 - val_loss: 0.5062 - val_acc: 0.7761
Epoch 359/2000
623/623 [==============================] - 0s 251us/step - loss: 0.4663 - acc: 0.7929 - val_loss: 0.5080 - val_acc: 0.7687
Epoch 360/2000
623/623 [==============================] - 0s 241us/step - loss: 0.4759 - acc: 0.7689 - val_loss: 0.5056 - val_acc: 0.7687
Epoch 361/2000
623/623 [==============================] - 0s 200us/step - loss: 0.4576 - acc: 0.7897 - val_loss: 0.5025 - val_acc: 0.7687
Epoch 362/2000
623/623 [==============================] - 0s 232us/step - loss: 0.4863 - acc: 0.7769 - val_loss: 0.5067 - val_acc: 0.7724
Epoch 363/2000
623/623 [=========================

623/623 [==============================] - 0s 290us/step - loss: 0.4408 - acc: 0.8026 - val_loss: 0.4959 - val_acc: 0.7724
Epoch 475/2000
623/623 [==============================] - 0s 284us/step - loss: 0.4407 - acc: 0.8026 - val_loss: 0.4945 - val_acc: 0.7724
Epoch 476/2000
623/623 [==============================] - 0s 267us/step - loss: 0.4504 - acc: 0.8026 - val_loss: 0.4932 - val_acc: 0.7724
Epoch 477/2000
623/623 [==============================] - 0s 254us/step - loss: 0.4500 - acc: 0.7978 - val_loss: 0.4966 - val_acc: 0.7724
Epoch 478/2000
623/623 [==============================] - 0s 225us/step - loss: 0.4468 - acc: 0.7994 - val_loss: 0.4957 - val_acc: 0.7724
Epoch 479/2000
623/623 [==============================] - 0s 271us/step - loss: 0.4507 - acc: 0.7913 - val_loss: 0.4935 - val_acc: 0.7724
Epoch 480/2000
623/623 [==============================] - 0s 252us/step - loss: 0.4385 - acc: 0.8074 - val_loss: 0.4980 - val_acc: 0.7799
Epoch 481/2000
623/623 [=========================

623/623 [==============================] - 0s 228us/step - loss: 0.4184 - acc: 0.8106 - val_loss: 0.5018 - val_acc: 0.7985
Epoch 593/2000
623/623 [==============================] - 0s 271us/step - loss: 0.3993 - acc: 0.8363 - val_loss: 0.5013 - val_acc: 0.8022
Epoch 594/2000
623/623 [==============================] - 0s 233us/step - loss: 0.4312 - acc: 0.8154 - val_loss: 0.5033 - val_acc: 0.7873
Epoch 595/2000
623/623 [==============================] - 0s 242us/step - loss: 0.4158 - acc: 0.8058 - val_loss: 0.5021 - val_acc: 0.7873
Epoch 596/2000
623/623 [==============================] - 0s 240us/step - loss: 0.4246 - acc: 0.8074 - val_loss: 0.4973 - val_acc: 0.7873
Epoch 597/2000
623/623 [==============================] - 0s 230us/step - loss: 0.4365 - acc: 0.7978 - val_loss: 0.5007 - val_acc: 0.7873
Epoch 598/2000
623/623 [==============================] - 0s 245us/step - loss: 0.4247 - acc: 0.8074 - val_loss: 0.5041 - val_acc: 0.7836
Epoch 599/2000
623/623 [=========================

623/623 [==============================] - 0s 221us/step - loss: 0.4002 - acc: 0.8250 - val_loss: 0.5282 - val_acc: 0.7873
Epoch 711/2000
623/623 [==============================] - 0s 231us/step - loss: 0.3952 - acc: 0.8315 - val_loss: 0.5222 - val_acc: 0.7873
Epoch 712/2000
623/623 [==============================] - 0s 230us/step - loss: 0.4010 - acc: 0.8202 - val_loss: 0.5274 - val_acc: 0.7873
Epoch 713/2000
623/623 [==============================] - 0s 254us/step - loss: 0.4082 - acc: 0.8138 - val_loss: 0.5205 - val_acc: 0.7836
Epoch 714/2000
623/623 [==============================] - 0s 240us/step - loss: 0.4060 - acc: 0.8186 - val_loss: 0.5175 - val_acc: 0.7761
Epoch 715/2000
623/623 [==============================] - 0s 216us/step - loss: 0.4054 - acc: 0.8186 - val_loss: 0.5266 - val_acc: 0.7873
Epoch 716/2000
623/623 [==============================] - 0s 219us/step - loss: 0.4011 - acc: 0.8299 - val_loss: 0.5187 - val_acc: 0.7836
Epoch 717/2000
623/623 [=========================

KeyboardInterrupt: 